In [ ]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

#servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome()

In [ ]:
# abrir a página index (entrar no site da busca jurídica)



In [ ]:
# importar a base de dados


In [ ]:


    # para cada processo (linha da tabela)


    # abrir a lista de cidades

    
    # selecionando o cidade

    
    # mudar para a nova aba

    
    # preencher o formulário com os dados de busca


    # clicar em pesquisar

    
    # confirmar a pesquisa

    
    # esperar o resultado da pesquisa e agir de acordo com o resultado
  